# Prediction Models (rfe)

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV

In [2]:
# Import utilities
sys.path.insert(0, '../')
from helpers import preprocessing
from helpers import sets
from helpers import metrics

## Preprocessing

### Import dataset

In [3]:
df = pd.read_csv("../../data_sets/ds3.csv")

### Encode target values

In [4]:
preprocessing.encoding(df)

### Test-Train split

In [5]:
from sklearn.model_selection import train_test_split

subset = sets.post["rfe"]

# Input data
X = df[subset]

# Target variable
y = df['y']

# Split dataset in training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Scale

In [6]:
pareto = preprocessing.ParetoScaler()
X_train = pareto.fit_transform(X_train)
X_test = pareto.transform(X_test)

## Decision Tree

In [7]:
from sklearn.tree import DecisionTreeClassifier

In [8]:
dt_model = DecisionTreeClassifier(criterion="gini", splitter="best")
dt_model.fit(X_train, y_train)

DecisionTreeClassifier()

In [9]:
y_pred = dt_model.predict(X_test)
metrics.metrics_report(dt_model, X, y, y_test, y_pred)

=== ACCURACY
Mean:		0.6857142857142857
STD:		0.26186146828319085
=== PRECISION
Mean:		0.8028571428571428
STD:		0.1883115091803895
=== RECALL
Mean:		0.78
STD:		0.20396078054371142
=== F1
Mean:		0.7571428571428571
STD:		0.266155973571088


## Logistic Regression

In [10]:
from sklearn.linear_model import LogisticRegression

In [11]:
lg_model = LogisticRegression(penalty="none")
lg_model.fit(X_train, y_train)

LogisticRegression(penalty='none')

In [12]:
y_pred = lg_model.predict(X_test)
metrics.metrics_report(lg_model, X, y, y_test, y_pred)

=== ACCURACY
Mean:		0.8
STD:		0.2138089935299395
=== PRECISION
Mean:		0.8
STD:		0.19436506316151003
=== RECALL
Mean:		0.85
STD:		0.3
=== F1
Mean:		0.8084848484848486
STD:		0.2487242010561114


## Naive Bayes

In [13]:
from sklearn.naive_bayes import GaussianNB

In [14]:
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)

GaussianNB()

In [15]:
y_pred = nb_model.predict(X_test)
metrics.metrics_report(nb_model, X, y, y_test, y_pred)

=== ACCURACY
Mean:		0.7428571428571428
STD:		0.20995626366712955
=== PRECISION
Mean:		0.7699999999999999
STD:		0.16
=== RECALL
Mean:		0.77
STD:		0.3026549190084311
=== F1
Mean:		0.7555555555555555
STD:		0.23726840560069584


## Support Vector Classifier (SVC)

In [16]:
from sklearn.svm import SVC

In [17]:
svc_model = SVC(kernel="rbf")
svc_model.fit(X_train, y_train)

SVC()

In [18]:
y_pred = svc_model.predict(X_test)
metrics.metrics_report(svc_model, X, y, y_test, y_pred)

=== ACCURACY
Mean:		0.6857142857142857
STD:		0.22857142857142856
=== PRECISION
Mean:		0.7366666666666667
STD:		0.2191904296369813
=== RECALL
Mean:		0.73
STD:		0.2821347195933177
=== F1
Mean:		0.7238095238095237
STD:		0.23534924435365892


## Artificial Neural Network (ANN)

In [19]:
import tensorflow as tf

2022-08-13 15:06:35.910157: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-13 15:06:35.910188: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [20]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, InputLayer

In [21]:
def build_model(n_hidden=1, n_neurons=6, learning_rate=3e-3, input_shape=X_train.shape[1]):
  model = Sequential()
  model.add(InputLayer(input_shape=input_shape))
  for layer in range(n_hidden):
    model.add(Dense(n_neurons, activation="relu"))
  model.add(Dense(1, activation='sigmoid'))
  optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
  model.compile(loss='binary_crossentropy', optimizer=optimizer,metrics = ['accuracy'])
  return model

In [22]:
from keras.wrappers.scikit_learn import KerasClassifier
ann_model = KerasClassifier(build_model)

/tmp/ipykernel_12833/2775760517.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ann_model = KerasClassifier(build_model)


In [23]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
from keras.callbacks import EarlyStopping

param_distribs = {
  "n_hidden": [0, 1, 2, 3],
  "n_neurons": np.arange(1, 10),
  "learning_rate": reciprocal(3e-4, 3e-2),
}
rnd_search_cv = RandomizedSearchCV(ann_model, param_distribs, n_iter=10,
cv=10)
rnd_search_cv.fit( X_train, y_train, batch_size = 32, epochs=100,                  
                  validation_split=0.05,  # using only for earlystopping
                  callbacks=[EarlyStopping(patience=5)])

2022-08-13 15:06:39.146836: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-13 15:06:39.146884: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-13 15:06:39.148001: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tron): /proc/driver/nvidia/version does not exist
2022-08-13 15:06:39.148404: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/100
1/1 [==============================] - 1s 1s/step - loss: 413.6326 - accuracy: 0.3913 - val_loss: 653.1848 - val_accuracy: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 54ms/step - loss: 412.1781 - accuracy: 0.3913 - val_loss: 650.8715 - val_accuracy: 0.0000e+00
Epoch 3/100
1/1 [==============================] - 0s 55ms/step - loss: 410.7267 - accuracy: 0.3913 - val_loss: 648.5628 - val_accuracy: 0.0000e+00
Epoch 4/100
1/1 [==============================] - 0s 49ms/step - loss: 409.2784 - accuracy: 0.3913 - val_loss: 646.2593 - val_accuracy: 0.0000e+00
Epoch 5/100
1/1 [==============================] - 0s 51ms/step - loss: 407.8331 - accuracy: 0.3913 - val_loss: 643.9607 - val_accuracy: 0.0000e+00
Epoch 6/100
1/1 [==============================] - 0s 57ms/step - loss: 406.3911 - accuracy: 0.3913 - val_loss: 641.6672 - val_accuracy: 0.0000e+00
Epoch 7/100
1/1 [==============================] - 0s 70ms/step - loss: 404.9522 - accuracy: 0.3913 - val_loss: 63

1/1 [==============================] - 0s 47ms/step - loss: 338.9427 - accuracy: 0.3913 - val_loss: 534.4742 - val_accuracy: 0.0000e+00
Epoch 57/100
1/1 [==============================] - 0s 48ms/step - loss: 337.6918 - accuracy: 0.3913 - val_loss: 532.4872 - val_accuracy: 0.0000e+00
Epoch 58/100
1/1 [==============================] - 0s 56ms/step - loss: 336.4448 - accuracy: 0.3913 - val_loss: 530.5062 - val_accuracy: 0.0000e+00
Epoch 59/100
1/1 [==============================] - 0s 43ms/step - loss: 335.2015 - accuracy: 0.3913 - val_loss: 528.5312 - val_accuracy: 0.0000e+00
Epoch 60/100
1/1 [==============================] - 0s 42ms/step - loss: 333.9620 - accuracy: 0.3913 - val_loss: 526.5623 - val_accuracy: 0.0000e+00
Epoch 61/100
1/1 [==============================] - 0s 53ms/step - loss: 332.7263 - accuracy: 0.3913 - val_loss: 524.5994 - val_accuracy: 0.0000e+00
Epoch 62/100
1/1 [==============================] - 0s 43ms/step - loss: 331.4944 - accuracy: 0.3913 - val_loss: 522.64

Epoch 11/100
1/1 [==============================] - 0s 45ms/step - loss: 3.7189 - accuracy: 0.8261 - val_loss: 2.0249e-21 - val_accuracy: 1.0000
Epoch 12/100
1/1 [==============================] - 0s 50ms/step - loss: 3.6163 - accuracy: 0.8696 - val_loss: 2.8066e-22 - val_accuracy: 1.0000
Epoch 13/100
1/1 [==============================] - 0s 50ms/step - loss: 3.5654 - accuracy: 0.8696 - val_loss: 3.9776e-23 - val_accuracy: 1.0000
Epoch 14/100
1/1 [==============================] - 0s 53ms/step - loss: 3.4449 - accuracy: 0.8696 - val_loss: 7.5452e-24 - val_accuracy: 1.0000
Epoch 15/100
1/1 [==============================] - 0s 46ms/step - loss: 3.3281 - accuracy: 0.8696 - val_loss: 1.8303e-24 - val_accuracy: 1.0000
Epoch 16/100
1/1 [==============================] - 0s 45ms/step - loss: 3.2126 - accuracy: 0.8696 - val_loss: 4.4938e-25 - val_accuracy: 1.0000
Epoch 17/100
1/1 [==============================] - 0s 43ms/step - loss: 3.0982 - accuracy: 0.8696 - val_loss: 1.1136e-25 - val_ac

1/1 [==============================] - 0s 41ms/step - loss: 840.0354 - accuracy: 0.3478 - val_loss: 1186.7936 - val_accuracy: 0.0000e+00
Epoch 16/100
1/1 [==============================] - 0s 37ms/step - loss: 837.7109 - accuracy: 0.3478 - val_loss: 1183.5132 - val_accuracy: 0.0000e+00
Epoch 17/100
1/1 [==============================] - 0s 34ms/step - loss: 835.3920 - accuracy: 0.3478 - val_loss: 1180.2397 - val_accuracy: 0.0000e+00
Epoch 18/100
1/1 [==============================] - 0s 30ms/step - loss: 833.0781 - accuracy: 0.3478 - val_loss: 1176.9739 - val_accuracy: 0.0000e+00
Epoch 19/100
1/1 [==============================] - 0s 30ms/step - loss: 830.7695 - accuracy: 0.3478 - val_loss: 1173.7161 - val_accuracy: 0.0000e+00
Epoch 20/100
1/1 [==============================] - 0s 33ms/step - loss: 828.4664 - accuracy: 0.3478 - val_loss: 1170.4652 - val_accuracy: 0.0000e+00
Epoch 21/100
1/1 [==============================] - 0s 32ms/step - loss: 826.1686 - accuracy: 0.3478 - val_loss: 

...

Epoch 18/100
1/1 [==============================] - 0s 32ms/step - loss: 188.0704 - accuracy: 0.3750 - val_loss: 360.1690 - val_accuracy: 0.5000
Epoch 19/100
1/1 [==============================] - 0s 28ms/step - loss: 187.0139 - accuracy: 0.3750 - val_loss: 356.5712 - val_accuracy: 0.5000
Epoch 20/100
1/1 [==============================] - 0s 33ms/step - loss: 185.9696 - accuracy: 0.3750 - val_loss: 353.0641 - val_accuracy: 0.5000
Epoch 21/100
1/1 [==============================] - 0s 31ms/step - loss: 184.9355 - accuracy: 0.3750 - val_loss: 349.6365 - val_accuracy: 0.5000
Epoch 22/100
1/1 [==============================] - 0s 30ms/step - loss: 183.9097 - accuracy: 0.3750 - val_loss: 346.2787 - val_accuracy: 0.5000
Epoch 23/100
1/1 [==============================] - 0s 28ms/step - loss: 182.8906 - accuracy: 0.3750 - val_loss: 342.9821 - val_accuracy: 0.5000
Epoch 24/100
1/1 [==============================] - 0s 27ms/step - loss: 181.8770 - accuracy: 0.3750 - val_loss: 339.7390 - val_ac

1/1 [==============================] - 0s 31ms/step - loss: 148.6933 - accuracy: 0.5000 - val_loss: 257.3565 - val_accuracy: 0.5000
Epoch 75/100
1/1 [==============================] - 0s 32ms/step - loss: 148.1472 - accuracy: 0.5000 - val_loss: 257.4329 - val_accuracy: 0.5000
Epoch 76/100
1/1 [==============================] - 0s 28ms/step - loss: 147.5988 - accuracy: 0.5000 - val_loss: 257.5424 - val_accuracy: 0.5000
Epoch 77/100
1/1 [==============================] - 0s 27ms/step - loss: 147.0489 - accuracy: 0.5000 - val_loss: 257.6767 - val_accuracy: 0.5000
Epoch 78/100
1/1 [==============================] - 0s 18ms/step - loss: 53.8203 - accuracy: 0.5000
Epoch 1/100
1/1 [==============================] - 1s 549ms/step - loss: 410.4533 - accuracy: 0.6250 - val_loss: 394.8713 - val_accuracy: 0.5000
Epoch 2/100
1/1 [==============================] - 0s 29ms/step - loss: 407.1807 - accuracy: 0.6250 - val_loss: 391.6135 - val_accuracy: 0.5000
Epoch 3/100
1/1 [===========================

1/1 [==============================] - 0s 27ms/step - loss: 243.5521 - accuracy: 0.6250 - val_loss: 228.7227 - val_accuracy: 0.5000
Epoch 53/100
1/1 [==============================] - 0s 28ms/step - loss: 240.2795 - accuracy: 0.6250 - val_loss: 225.4649 - val_accuracy: 0.5000
Epoch 54/100
1/1 [==============================] - 0s 31ms/step - loss: 237.0070 - accuracy: 0.6250 - val_loss: 222.2071 - val_accuracy: 0.5000
Epoch 55/100
1/1 [==============================] - 0s 27ms/step - loss: 233.7344 - accuracy: 0.6250 - val_loss: 218.9492 - val_accuracy: 0.5000
Epoch 56/100
1/1 [==============================] - 0s 32ms/step - loss: 230.4618 - accuracy: 0.6250 - val_loss: 215.6914 - val_accuracy: 0.5000
Epoch 57/100
1/1 [==============================] - 0s 29ms/step - loss: 227.1892 - accuracy: 0.6250 - val_loss: 212.4336 - val_accuracy: 0.5000
Epoch 58/100
1/1 [==============================] - 0s 30ms/step - loss: 223.9167 - accuracy: 0.6250 - val_loss: 209.1758 - val_accuracy: 0.500

1/1 [==============================] - 0s 26ms/step - loss: 0.6947 - accuracy: 0.3846 - val_loss: 0.6937 - val_accuracy: 0.0000e+00
Epoch 9/100
1/1 [==============================] - 0s 30ms/step - loss: 0.6933 - accuracy: 0.3846 - val_loss: 0.6871 - val_accuracy: 1.0000
Epoch 10/100
1/1 [==============================] - 0s 28ms/step - loss: 0.6918 - accuracy: 0.6154 - val_loss: 0.6799 - val_accuracy: 1.0000
Epoch 11/100
1/1 [==============================] - 0s 29ms/step - loss: 0.6902 - accuracy: 0.6154 - val_loss: 0.6725 - val_accuracy: 1.0000
Epoch 12/100
1/1 [==============================] - 0s 27ms/step - loss: 0.6885 - accuracy: 0.6154 - val_loss: 0.6648 - val_accuracy: 1.0000
Epoch 13/100
1/1 [==============================] - 0s 31ms/step - loss: 0.6869 - accuracy: 0.6154 - val_loss: 0.6569 - val_accuracy: 1.0000
Epoch 14/100
1/1 [==============================] - 0s 30ms/step - loss: 0.6853 - accuracy: 0.6154 - val_loss: 0.6489 - val_accuracy: 1.0000
Epoch 15/100
1/1 [=====

RandomizedSearchCV(cv=10,
                   estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7f1e6e365cd0>,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f1e6e37d790>,
                                        'n_hidden': [0, 1, 2, 3],
                                        'n_neurons': array([1, 2, 3, 4, 5, 6, 7, 8, 9])})

In [24]:
(rnd_search_cv.best_params_, rnd_search_cv.best_score_)

({'learning_rate': 0.02642811201539949, 'n_hidden': 3, 'n_neurons': 5},
 0.7000000059604645)

In [25]:
ann = rnd_search_cv.best_estimator_.model

In [26]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 10)

Epoch 1/10


2022-08-13 15:11:19.890130: W tensorflow/core/common_runtime/forward_type_inference.cc:231] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected compatible input types, but input 1:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_BOOL
    }
  }
}
 is neither a subtype nor a supertype of the combined inputs preceding it:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_LEGACY_VARIANT
    }
  }
}

	while inferring type of node 'binary_crossentropy/cond/output/_10'


1/1 [==============================] - 1s 669ms/step - loss: 0.6526 - accuracy: 0.6429
Epoch 2/10
1/1 [==============================] - 0s 5ms/step - loss: 0.6526 - accuracy: 0.6429
Epoch 3/10
1/1 [==============================] - 0s 6ms/step - loss: 0.6526 - accuracy: 0.6429
Epoch 4/10
1/1 [==============================] - 0s 5ms/step - loss: 0.6525 - accuracy: 0.6429
Epoch 5/10
1/1 [==============================] - 0s 6ms/step - loss: 0.6525 - accuracy: 0.6429
Epoch 6/10
1/1 [==============================] - 0s 7ms/step - loss: 0.6525 - accuracy: 0.6429
Epoch 7/10
1/1 [==============================] - 0s 5ms/step - loss: 0.6524 - accuracy: 0.6429
Epoch 8/10
1/1 [==============================] - 0s 5ms/step - loss: 0.6524 - accuracy: 0.6429
Epoch 9/10
1/1 [==============================] - 0s 5ms/step - loss: 0.6523 - accuracy: 0.6429
Epoch 10/10
1/1 [==============================] - 0s 5ms/step - loss: 0.6523 - accuracy: 0.6429


In [27]:
metrics.ann_metrics_report(ann, X, y, 5)

1/1 [==============================] - 0s 21ms/step
=== ACCURACY
MEAN:		0.6285714285714286
STD:		0.45714285714285713
